# Discretization

In this section, we start to actually describe how we want to implement the solver for the minimal surface problem. We discuss two main classes which are used to implement the problem in Python and which are also used in the eventual solver implementation.

## Grid

We begin by describing a simple class called `Grid`. This class is used to represent the 3-torus $\mathbb{T}^3$. Most of the implementation is the same as implementing a discrete grid over the unit cube $[0, 1]^3$ but the periodic boundary of the torus is taken into account when accessing different points of the shape.

The whole implementation is stored in `src/Grid.py` but here we will demonstrate some of the most common operations.

In [1]:
from src.grid import Grid

In [2]:
grid = Grid(resolution=8)

In [3]:
grid.indices_flat

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       ...,
       [7, 7, 5],
       [7, 7, 6],
       [7, 7, 7]], shape=(512, 3))

In [9]:
# Get a specific position with some index.
print(grid.index_to_position([4, 1, 2]))

# Get a specific index with some position
print(grid.position_to_index([0.5, 0.25, 0.125]))

[0.5   0.125 0.25 ]
[4 2 1]


In [21]:
# Wrapping around works as expected
print(grid.index_to_position([20, 9, 18]))
print(grid.position_to_index([1.5, 1.25, 2.128]))

# Negative values also
print(grid.index_to_position([-1, 0, 0]))
print(grid.position_to_index([-2.125, -1.25, 1.25]))

[0.5   0.125 0.25 ]
[4 2 1]
[0.875 0.    0.   ]
[7 6 2]
